In [1]:
from bs4 import BeautifulSoup
import requests
import re
import urllib.request
from urllib.request import urlopen
import os

In [2]:
import regex as re
from langdetect import detect
import csv
import pandas as pd

In [3]:
#soup = BeautifulSoup(open("Best Books Ever (53139 books).html"), features="lxml") 
#not convenient.. let's try to load the page

In [9]:
#Exercise 1.1

web_page = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1")

def get_url(url):
    req = urllib.request.Request(url)
    return urllib.request.urlopen(req)

def listToString(s):  
    
    # initialize an empty string 
    str1 = "\n"
    # return string   
    return (str1.join(s))

In [44]:
page = 151
books_urls = []
while (page>=151 and page <= 300):
    web_page = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever?page={}".format(page))
    soup = BeautifulSoup(web_page.content, features="lxml")
    for link in soup.find_all('a', class_='bookTitle', itemprop = 'url', href = True):
        url = link.get('href')
        url = "https://www.goodreads.com/"+url
        books_urls.append(url)
    #book_links = print_book_links(soup)
    str_book_links = listToString(books_urls)
    page+=1

In [4]:
#str_book_links #it looks orrible now.. but the file will be written correctly

In [45]:
#save all links found into a file.txt
f = open("BookLinks.txt", "w")
f.write(str_book_links)
f.close()

In [14]:
#Exercise 1.2

page = 233
htmls = []
while (page>= 233 and page <= 300):
    
    directory = "HTMLSources_Page"+str(page)
    parent_dir = "/home/lorenzo/Documents/Università/1° Term/Algorithmic Methods of Data Mining/LAB/Homeworks/HW3"
    path = os.path.join(parent_dir, directory)
    os.mkdir(path)
    
    web_page = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever?page={}".format(page))
    soup = BeautifulSoup(web_page.content, features="lxml")
    
    i = 1
    
    for link in soup.find_all('a', class_='bookTitle', itemprop = 'url', href = True):
        url = link.get('href')
        url = "https://www.goodreads.com/"+url
        html = urllib.request.urlopen(url).read()
        html_formatted = html.decode("utf-8")
        
        namefile = "Book"+str(i)+"_"+"Page"+str(page)
        with open(os.path.join(path,namefile), 'w') as f:
            f.write(str(html_formatted))
            f.close()
        i += 1
    
    page += 1

In [8]:
#Exercise 1.3

import codecs

page = 151

i = 1 #book_id

while (page <= 151):
    #let's move to the correct directory --> the directory associated to the current page
    directory = "HTMLSources_Page"+str(page)
    parent_dir = "/home/lorenzo/Documents/Università/1° Term/Algorithmic Methods of Data Mining/LAB/Homeworks/HW3"
    path = os.path.join(parent_dir, directory)
    os.chdir(path)
    
    while (i <= 100):#each page contains 100 books
    
        filename = "Book"+str(i)+"_Page"+str(page)
        #print(filename)
    
        #now I need to read the stored locally html files
        file = codecs.open(filename, 'r', 'utf-8')
        document = BeautifulSoup(file.read(), features="lxml") #"document" contains the html text of the corresp file
        
        print("Title: ",document.find_all("h1", id="bookTitle", class_="gr-h1 gr-h1--serif", itemprop="name")[0].contents[0])
        print("Series: ",document.find_all("a", class_="greyText")[0].contents[0])
        print("Author: ",document.find_all("span", itemprop="name")[0].contents[0])
        print("Rating Value: ",document.find_all("span", itemprop="ratingValue")[0].contents[0])
        print("Rating Count: ",document.find_all("a", class_="gr-hyperlink", href="#other_reviews")[0].contents[2])
        print("Plot: ", ' '.join([c for c in document.find_all('div', id="description")[0].contents[1].contents if isinstance(c, str)]))
        print("Number of Pages: ",document.find_all("span", itemprop="numberOfPages")[0].contents[0])
        print("Publishing Date: ", document.find_all("div", class_="row")[1].contents[0])
        #print("Characters: ", document.find_all("a", href = href.split("/")) if href[0] == "characters") ###WRONG! 
        #SETTING
        
        #description = ' '.join([c for c in soup.find_all('div', id="description")[0].contents[1].contents if isinstance(c, str)])
        
        i+=1 #increment book_id
    
    page += 1 #increment page
        
    os.chdir('../') #back to the parent working directory

Title:  
      The Greek Myths: 1

Series:  
        (The Greek Myths #1)

Author:  Robert Graves
Rating Value:  
  4.08

Rating Count:  
  1,236
  ratings

Plot:  In a work that has become a classic reference book for both the serious scholar and the casual inquirer, Graves retells the adventures of the important gods and heroes worshipped by the ancient Greeks. Each entry provides a full commentary which examines problems of interpretation in both historical and anthropological terms, and in light of contemporary research.
Number of Pages:  370 pages
Publishing Date:  
            Published
        December 1st 1990
         by Penguin Books

            
Title:  
      The Measure of a Man: A Spiritual Autobiography

Series:  Edit Details
Author:  Sidney Poitier
Rating Value:  
  3.72

Rating Count:  
  10,116
  ratings

Plot:  In this luminous memoir, a true American icon looks back on his celebrated life and career.  His body of work is arguable the most morally significant in cin

IndexError: list index out of range

In [56]:
#Exercise 1.3



from langdetect import detect
from textblob import TextBlob
#function to parse the html pages
def parse_pages(parent_dir):

    '''
    function to parse the downloded pages and take the infos we need
    input: parent_dir with the downloaded pages
    output: .tsv files
    '''
    article = [] #to create .tsv file
    parent_dir = "/home/lorenzo/Documents/Università/1° Term/Algorithmic Methods of Data Mining/LAB/Homeworks/HW3"
    
    p = 151        #pages, starting from 151
    n = 1
    while (p<=151): #we start opening the folder related to a specific page
        directory = "HTMLSources_Page"+str(p) #directory of the page
        path = os.path.join(parent_dir, directory) #create path
        os.chdir(path)
        
        book_id = 1
        
        #book id per page, starting from 1
        while(book_id <= 10):
            
            book = "Book"+str(book_id)+"_"+"Page"+str(p)
            file = open(book, 'r')
            html_doc =  BeautifulSoup(file.read(), features="lxml")
    
            #Infos we want to know
    
            bookTitle = html_doc.find_all('h1')[0].contents[0]
            bookTitle = bookTitle.split('\n')[1].strip()
            article.append(bookTitle)
    
    
            bookAuthors = html_doc.find_all('span', itemprop="name")[0].contents[0]
            article.append(bookAuthors)
    
            bookSeries = html_doc.find_all('a', class_="greyText")[0].contents[0]
            #print(bookSeries)
            if bookSeries == 'Edit Details':
                bookSeries = ''
                article.append(bookSeries)
            else:
                bookSeries = bookSeries.split('\n')[1].strip()
                article.append(bookSeries)
            #print(bookSeries)
            
            
    
            ratingValue = html_doc.find_all('span', itemprop="ratingValue")[0].contents[0]
            ratingValue = ratingValue.split('\n')[1].strip()
            article.append(ratingValue)
    
            ratingCount = html_doc.find_all('a', href="#other_reviews")[0].contents[2]
            ratingCount = ratingCount.split('\n')[1].strip()
            article.append(ratingCount)
    
            reviewCount = html_doc.find_all('a', href="#other_reviews")[1].contents[2]
            reviewCount = reviewCount.split('\n')[1].strip()
            article.append(reviewCount)
    
            if (html_doc.find_all('div', {id : "description"})):
                Plot = ' '.join([c for c in html_doc.find_all('div', id="description")[0].contents[1].contents if isinstance(c, str)])
            else:
                Plot = "   "
            article.append(Plot)
            
            if (html_doc.find_all('span', {'itemprop' : "numberOfPages"})):      
                NumberofPages = html_doc.find_all('span', itemprop="numberOfPages")[0].contents[0]
                NumberofPages = NumberofPages.split(' ')[0]
            else:
                NumberofPages = ''
            article.append(NumberofPages)
            
            if (html_doc.find_all('div', {'class' : "row"})[1].contents[0]):
                PublishingDate = html_doc.find_all('div', class_="row")[1].contents[0]
                if (len(PublishingDate.split('\n')) >= 3):
                    PublishingDate = PublishingDate.split('\n')[2].strip()
                else:
                    PublishingDate = ""
            else:
                PublishingDate = ""    
            article.append(PublishingDate)
            
            if html_doc.find_all('a', {'href' : re.compile(r'/characters')}):
                Characters = ','.join([item.text for item in  html_doc.find_all('a', href = re.compile(r'/characters'))])
            else:
                Characters = ""
            article.append(Characters)
            
            if html_doc.find_all('a', {'href' : re.compile(r'/places')}):
                Setting = ' '.join([item.text for item in  html_doc.find_all('a', href = re.compile(r'/places'))]) 
            else:
                Setting = ""
            article.append(Setting)
            
            Url = html_doc.find_all('link')[0]
            #Url = html_doc.get('href')
            #Url = "https://www.goodreads.com/"+Url
            print(Url)
            article.append(Url)
    
            #Url = html_doc.find_all('span', itemprop="name")[0].contents[0]
            #print(Url)
            lang = TextBlob(Plot)

    
            if lang.detect_language() == 'eng' or Plot=="   ": 
                path = "/home/lorenzo/Documents/Università/1° Term/Algorithmic Methods of Data Mining/LAB/Homeworks/HW3/Articles"
            #name_article = "articles.tsv"
                name_article = "article_"+str(n)+".tsv"
    
                with open(os.path.join(path,name_article), 'w', newline='') as f_output:
                    tsv_output = csv.writer(f_output, delimiter='\t')
                    tsv_output.writerow(article)
                    n += 1
                    article.clear()
                #article.append()
                
                book_id +=1
        p +=1

#####################################################        
article = [] #to create .tsv file
parent_dir = "/home/lorenzo/Documents/Università/1° Term/Algorithmic Methods of Data Mining/LAB/Homeworks/HW3"
parse_pages(parent_dir)

<link href="https://www.goodreads.com/book/show/510569.The_Greek_Myths" rel="canonical"/>
<link href="https://www.goodreads.com/book/show/91563.The_Measure_of_a_Man" rel="canonical"/>
<link href="https://www.goodreads.com/book/show/52917242-silent-pantheon" rel="canonical"/>
<link href="https://www.goodreads.com/book/show/4224060-the-element" rel="canonical"/>
<link href="https://www.goodreads.com/book/show/26831901-the-person-controller" rel="canonical"/>
<link href="https://www.goodreads.com/book/show/169878.The_Grand_Tour" rel="canonical"/>
<link href="https://www.goodreads.com/book/show/19398.Complete_Shorter_Fiction" rel="canonical"/>
<link href="https://www.goodreads.com/book/show/1420762.The_Rosary" rel="canonical"/>
<link href="https://www.goodreads.com/book/show/289643.Sunrise" rel="canonical"/>
<link href="https://www.goodreads.com/book/show/16856.Flappers_and_Philosophers" rel="canonical"/>


In [44]:
pd.read_csv("/home/lorenzo/Documents/Università/1° Term/Algorithmic Methods of Data Mining/LAB/Homeworks/HW3/Articles/article_100.tsv",delimiter='\t',encoding='utf-8')

,Finding Perfect,Susan Mallery,(Fool's Gold #3),4.07,"11,648",512,Unnamed: 6,346,August 31st 2010,"Pia O’Brian,Raoul Moreno","Fool's Gold, California"


# Exercise 2

In [6]:
import nltk #Natural Language ToolKit
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/lorenzo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
article = pd.read_csv("/home/lorenzo/Documents/Università/1° Term/Algorithmic Methods of Data Mining/LAB/Homeworks/HW3/Articles/article_1.tsv",delimiter='\t',encoding='utf-8')

In [12]:
article['Plot'].apply(word_tokenize)

KeyError: 'Plot'

In [28]:
#remove useless charachters from all TSVs
i = 1
n = 1

dataset = []

while i<=15:
    article = pd.read_csv("/home/lorenzo/Documents/Università/1° Term/Algorithmic Methods of Data Mining/LAB/Homeworks/HW3/Articles/article_"+str(i)+".tsv",delimiter='\t',encoding='utf-8')
    article = article.columns
    tokens_list = []
    for k in range(len(article)):
        removeSpecialChars = "".join(c for c in article[k] if c not in ("/","<",">","|","`","~","-","=","_","+","?", ",", ".", ";", ":", "!", "@","#","$","%","^","&","*","(",")","[","]","{","}"))
        words = removeSpecialChars
        tokens = nltk.word_tokenize(words)
        def_string = ' '.join(el for el in tokens)
        tokens_list.append(def_string)
    
    path = "/home/lorenzo/Documents/Università/1° Term/Algorithmic Methods of Data Mining/LAB/Homeworks/HW3/Articles"
    name_article = "article_"+str(i)+".tsv"
    
    with open(os.path.join(path,name_article), 'w', newline='') as f_output:
        tsv_output = csv.writer(f_output, delimiter='\t')
        tsv_output.writerow(tokens_list)
    
    tokens_list.clear()
    
    i += 1

#print(dataset)

In [ ]:
# 2.1

while i<=15:
    article = pd.read_csv("/home/lorenzo/Documents/Università/1° Term/Algorithmic Methods of Data Mining/LAB/Homeworks/HW3/Articles/article_"+str(i)+".tsv",delimiter='\t',encoding='utf-8')
    article = article.columns
    tokens_list = []
